Step 1: Combine All Datasets

In [50]:
import pandas as pd
import numpy as np
import pymysql

# Define file-to-state mapping
datasets = {
    "Telangana_bus_details.csv": "Telangana",
    "kerala_bus_details.csv": "Kerala",
    "Andhra_bus_details.csv": "Andhra",
    "Kadamba_bus_details.csv": "Kadamba",
    "Rajasthan_bus_details.csv": "Rajasthan",
    "SBengal_bus_details.csv": "South Bengal",
    "Himachal_bus_details.csv": "Himachal",
    "Assam_bus_details.csv": "Assam",
    "Jammu_bus_details.csv": "Jammu",
    "WBengal_bus_details.csv": "West Bengal",
    "Bihar_bus_details.csv": "Bihar"
}

# Combine all datasets
dataframes = []
for file, state in datasets.items():
    try:
        df = pd.read_csv(file)
        df.insert(0, "State", state)  # Add state column
        dataframes.append(df)
    except FileNotFoundError:
        print(f"File {file} not found. Skipping...")
    except Exception as e:
        print(f"Error processing {file}: {e}")

Overall_df = pd.concat(dataframes, ignore_index=True)

Step 2: Clean Data

In [51]:
# Ensure specific columns are numeric
Overall_df['Price'] = pd.to_numeric(Overall_df['Price'], errors='coerce').fillna(0)
Overall_df['Star_Rating'] = pd.to_numeric(Overall_df['Star_Rating'], errors='coerce').fillna(0)

# Fill NaNs with default values
columns_with_defaults = {
    'Bus_Name': 'Unknown Bus',
    'Route_Name': 'Unknown Route',
    'Departing_Time': 'Unknown Start Time',
    'Reaching_Time': 'Unknown End Time',
    'Seat_Availability': 'Unknown Seats',
    'Duration': 'Unknown Duration',
    'Bus_Type': 'Unknown Type'
}
Overall_df.fillna(columns_with_defaults, inplace=True)

# Replace remaining NaNs with None
Overall_df.replace({np.nan: None}, inplace=True)

Step 3: Save the cleaned data to a CSV

In [52]:
path = r"C:\Users\jeyaj\Downloads\GUVI\RedBus Project\Overall_Bus_details.csv"
Overall_df.to_csv(path, index=False)

Step 4: Connect to MySQL

In [53]:
conn = pymysql.connect(host='127.0.0.1', user='root', passwd='Mysql@17072000')
cur = conn.cursor()

# Create database if it doesn't exist
cur.execute("CREATE DATABASE IF NOT EXISTS Overall_Bus_details")
cur.execute("USE Overall_Bus_details")

0

Step 5: Create Table

In [54]:
cur.execute('''
    CREATE TABLE IF NOT EXISTS Bus_details (
        ID INT AUTO_INCREMENT PRIMARY KEY,
        State VARCHAR(255) NOT NULL,
        Route_Name VARCHAR(255) NOT NULL,
        Route_Link VARCHAR(255) NULL,
        Bus_Name VARCHAR(255) NOT NULL,
        Bus_Type VARCHAR(255) NOT NULL,
        Start_Time VARCHAR(255) NOT NULL,
        Total_Duration VARCHAR(255) NOT NULL,
        End_Time VARCHAR(255) NOT NULL,
        Star_Rating FLOAT NULL,
        Price FLOAT NULL,
        Seats_Available VARCHAR(255) NOT NULL
    )
''')
print("Table Created")

Table Created


Step 6: Insert Data

In [55]:
Insert_data = '''
    INSERT INTO Bus_details (
        State,
        Route_Name,
        Route_Link,
        Bus_Name,
        Bus_Type,
        Start_Time,
        Total_Duration,
        End_Time,
        Star_Rating,
        Price,
        Seats_Available
    ) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
'''

Data = Overall_df.values.tolist()

try:
    # Insert data row by row
    for row in Data:
        try:
            cur.execute(Insert_data, row)
        except Exception as e:
            print(f"Error inserting row {row}: {e}")
            conn.rollback()
    conn.commit()
    print("Data Inserted Successfully")
except Exception as e:
    conn.rollback()
    print(f"Error during insertion: {e}")
finally:
    conn.close()

Data Inserted Successfully
